In [9]:
import json
import os

def reduce_to_level1_jsonl_keep_format(input_path, output_path):
    count = 0
    with open(input_path, 'r', encoding='utf-8') as fin, \
         open(output_path, 'w', encoding='utf-8') as fout:
        for line in fin:
            line = line.strip()
            if not line:
                continue
            try:
                obj = json.loads(line)
            except json.JSONDecodeError as e:
                print(f"Skipping invalid JSON line in {input_path}: {e}")
                continue

            arg1 = obj.get("Arg1")
            arg2 = obj.get("Arg2")
            # 可能你的 Label 字段名为 "conn1_sense1"
            label_raw = obj.get("Label")
            if label_raw is None:
                label_raw = obj.get("conn1_sense1")
            if label_raw is None:
                # 没有标签则跳过
                continue

            # 取一级标签
            if "." in label_raw:
                lvl1 = label_raw.split(".")[0]
            else:
                lvl1 = label_raw

            new_obj = {
                "Arg1": arg1,
                "Arg2": arg2,
                "Label": lvl1
            }
            fout.write(json.dumps(new_obj, ensure_ascii=False) + "\n")
            count += 1

    print(f"Finished processing {input_path} → {output_path}, total lines: {count}")

if __name__ == "__main__":
    files = [
        ("pdtb3_train.json", "pdtb3_train_level1.jsonl"),
        ("pdtb3_dev.json",   "pdtb3_dev_level1.jsonl"),
        ("pdtb3_test.json",  "pdtb3_test_level1.jsonl")
    ]
    for inp, outp in files:
        if os.path.exists(inp):
            reduce_to_level1_jsonl_keep_format(inp, outp)
        else:
            print(f"File not found: {inp}")


Finished processing pdtb3_train.json → pdtb3_train_level1.jsonl, total lines: 17085
Finished processing pdtb3_dev.json → pdtb3_dev_level1.jsonl, total lines: 1615
Finished processing pdtb3_test.json → pdtb3_test_level1.jsonl, total lines: 1474


In [1]:
import json
import os

# 映射一级标签到字母
label_to_letter = {
    "Comparison": "A",
    "Contingency": "B",
    "Expansion":   "C",
    "Temporal":    "D"
}

def convert_to_prompt_array(input_path, output_path):
    records = []
    with open(input_path, 'r', encoding='utf-8') as fin:
        for line in fin:
            line = line.strip()
            if not line:
                continue
            try:
                obj = json.loads(line)
            except json.JSONDecodeError as e:
                print(f"Skipping invalid JSON line in {input_path}: {e}")
                continue

            arg1 = obj.get("Arg1", "").replace("\n", " ")
            arg2 = obj.get("Arg2", "").replace("\n", " ")
            label = obj.get("Label", "")

            if label not in label_to_letter:
                continue

            letter = label_to_letter[label]

            rec = {
                "instruction": "What is the discourse relation between Arg 1 and Arg 2?. Only output one letter: A, B, C, or D.",
                "input": f"Arg1: {arg1}\nArg2: {arg2}\n\nA. Comparison\nB. Contingency\nC. Expansion\nD. Temporal",
                "output": letter
            }
            records.append(rec)

    with open(output_path, 'w', encoding='utf-8') as fout:
        json.dump(records, fout, ensure_ascii=False, indent=2)

    print(f"Converted {input_path} → {output_path}, total records: {len(records)}")

if __name__ == "__main__":
    files = [
        ("pdtb3_train_level1.jsonl", "pdtb3_train_instruction.json"),
        # ("pdtb3_dev_level1.jsonl",   "pdtb3_dev_instruction.json"),
        # ("pdtb3_test_level1.jsonl",  "pdtb3_test_instruction.json")
    ]
    for inp, outp in files:
        if os.path.exists(inp):
            convert_to_prompt_array(inp, outp)
        else:
            print(f"File not found: {inp}")


Converted pdtb3_train_level1.jsonl → pdtb3_train_instruction.json, total records: 17115


In [ ]:
import os
import json
from tqdm import tqdm
from openai import OpenAI

# 初始化 DeepSeek 客户端
client = OpenAI(
    api_key="sk-dc0d1289f0c2497cbccd7cadce9e4079",
    base_url="https://api.deepseek.com"
)

# ====== 配置路径 ======
INPUT_FILE = "./output.jsonl"      # 输入数据集
OUTPUT_FILE = "pdtb3_augmented_output.jsonl"  # 输出增强结果
BATCH_SIZE = 1  

# ====== 数据截取范围======
START_IDX = 31
END_IDX = 100
# Expansion的100-200
# INSTRUCTION = (
#     "请修改下面PDTB3中的论元对（Arg1, Arg2）的隐式关系为Temporal"
#     "必须修改Arg1和Arg2，保证他们的关系发生了变化后，再根据新关系修改Label，保持原格式JSON输出，不要输出任何多余说明或文字。"
#     "Label是Arg1和Arg2之间的关系，共有Temporal、Expansion、Comparison和Contingenc四种，修改后的Arg1和Arg2要契合Temporal且和其他三个有显著区分度"
#     "不必严格保持语义，只需保持领域和话题一致，并且有原句子的风格。"
#     "不能添加显式连接词（如 later, then, subsequently 等）。"
#     "必须保持两个论元之间的关系仍然是隐式。"
# )

# INSTRUCTION = (
#     "请根据Arg1得到Arg2，保证Arg1和Arg2直接的关系是Temporal，必须要有明确的时间先后的关系，可以对Arg1做最低限度的适当修改以适应这种关系"    
#     "Label是Arg1和Arg2之间的关系，共有Temporal、Expansion、Comparison和Contingency四种，你生成的Arg2要契合所给的Label且和其他三种有较大区分"
#     "需保持领域和话题一致，并且有原句子的风格。"
#     "不能添加显式连接词（如 later, then, subsequently 等）。"
#     "必须保持两个论元之间的关系仍然是隐式。"
#     "保持原格式JSON输出，即Arg1+Arg2+Label，不要输出任何多余说明或文字。"
# )

INSTRUCTION = (
    "请根据Arg1生成Arg2。确保Arg1和Arg2之间的关系是 Temporal（时间关系）。"
    "两者必须存在明确的时间先后或同时发生的关系。"
    "Arg1和Arg2关系仅体现时间顺序或同时性，不涉及因果、对比或补充。"
    "可以在保持话题领域和风格一致的前提下，对 Arg1 做有限的调整，以使这种时间关系自然成立且让Arg1和Arg2都描述事件或状态。"
    "你必须移除因果关系，如果一个事件导致另一个事件，则仍然属于因果（Contingency），不能标为Temporal即使存在时间上的关系。"
    "不要添加显式时间连接词（如 later, then, subsequently 等）。"
    "必须保持两个论元之间的关系是隐式的。"
    "保持输出格式为 JSON，仅包含 Arg1、Arg2、Label 字段，不输出任何额外文字。"
)

# INSTRUCTION = (
#     "请修改下面PDTB3中的论元对（Arg1, Arg2）的隐式关系为时间（Temporal），他们现在的关系是因果（Contingency）"
#     "两者必须存在明确的时间先后或同时发生的关系。"
#     "Arg1和Arg2关系仅体现时间顺序或同时性，不涉及因果、对比或补充。"
#     "你必须移除因果关系，如果一个事件导致另一个事件，则仍然属于因果（Contingency），不能标为Temporal。"
#     "请保持两个论元的领域和话题不变，不破坏原句行文风格。"
#     "不要添加显式时间连接词（如 later, then, subsequently 等）。"
#     "必须保持两个论元之间的关系是隐式的。"
#     "保持输出格式为 JSON，仅包含 Arg1、Arg2、Label 字段，不输出任何额外文字。"
# )
# ====== 定义函数 ======
def augment_sample(sample):
    """调用 DeepSeek API 修改单条样本"""
    prompt = f"{INSTRUCTION}\n原始数据：\n{json.dumps(sample, ensure_ascii=False)}"

    response = client.chat.completions.create(
        model="deepseek-reasoner",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that outputs pure JSON only."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.8,
        stream=False
    )

    result = response.choices[0].message.content.strip()

    try:
        return json.loads(result)
    except json.JSONDecodeError:
        try:
            json_part = result[result.find("{"): result.rfind("}") + 1]
            return json.loads(json_part)
        except Exception:
            print("⚠️ 无法解析输出，跳过样本：", result)
            return None


# ====== 主程序 ======
def main():
    with open(INPUT_FILE, "r", encoding="utf-8") as f_in:
        lines = [line.strip() for line in f_in if line.strip()]
    
    total = len(lines)
    print(f"📄 数据总量：{total} 条")
    start = max(0, START_IDX - 1)
    end = min(END_IDX, total)

    print(f"🚀 本次将处理第 {START_IDX} ~ {END_IDX} 条样本（共 {end - start} 条）")

    with open(OUTPUT_FILE, "w", encoding="utf-8") as f_out:
        for i, line in enumerate(tqdm(lines[start:end], desc="数据增强中..."), start=start + 1):
            sample = json.loads(line)
            augmented = augment_sample(sample)
            if augmented:
                json.dump(augmented, f_out, ensure_ascii=False)
                f_out.write("\n")

    print(f"\n✅ 数据增强完成，输出文件：{OUTPUT_FILE}")
 
 
if __name__ == "__main__":
    main()


📄 数据总量：100 条
🚀 本次将处理第 11 ~ 50 条样本（共 40 条）


数据增强中...:  50%|█████     | 20/40 [47:46<47:46, 143.31s/it]  


KeyboardInterrupt: 

In [4]:
import json
from collections import Counter

# 你的数据文件路径
file_path = "/data/ykyang/datasets/pdtb3_T5/pdtb3_train_level1.jsonl"  # 支持每行一个json或整个json列表

counts = Counter()

with open(file_path, 'r', encoding='utf-8') as f:
    first_char = f.read(1)
    f.seek(0)
    if first_char == '[':
        # JSON数组格式
        data = json.load(f)
        for item in data:
            label = item.get("Label")
            if label:
                counts[label] += 1
    else:
        # JSON行格式
        for line in f:
            if not line.strip():
                continue
            item = json.loads(line)
            label = item.get("Label")
            if label:
                counts[label] += 1

# 打印结果
print("=== 四个类别统计结果 ===")
for label in ["Contingency", "Comparison", "Temporal", "Expansion"]:
    print(f"{label}: {counts[label]}")


=== 四个类别统计结果 ===
Contingency: 5896
Comparison: 1830
Temporal: 1418
Expansion: 7941


In [6]:
import json
import os

# 输入数据路径
input_path = "/data/ykyang/datasets/pdtb3_T5/pdtb3_train_level1.jsonl"

# 输出目录
output_dir = "pdtb3_split_level1"
os.makedirs(output_dir, exist_ok=True)

# 四个类别的缓存
data_by_label = {
    "Contingency": [],
    "Comparison": [],
    "Temporal": [],
    "Expansion": []
}

# 读取数据（支持JSON行或JSON数组）
with open(input_path, 'r', encoding='utf-8') as f:
    first_char = f.read(1)
    f.seek(0)
    if first_char == '[':
        # JSON数组
        data = json.load(f)
        for item in data:
            label = item.get("Label")
            if label in data_by_label:
                data_by_label[label].append(item)
    else:
        # JSON行
        for line in f:
            if not line.strip():
                continue
            item = json.loads(line)
            label = item.get("Label")
            if label in data_by_label:
                data_by_label[label].append(item)

# 保存拆分结果
for label, items in data_by_label.items():
    output_file = os.path.join(output_dir, f"{label}.json")
    with open(output_file, 'w', encoding='utf-8') as f:
        for item in items:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    print(f"{label}: {len(items)} 条样本已保存到 {output_file}")

print("\n✅ 数据集已按类别拆分完成！")


Contingency: 5896 条样本已保存到 pdtb3_split_level1/Contingency.json
Comparison: 1830 条样本已保存到 pdtb3_split_level1/Comparison.json
Temporal: 1418 条样本已保存到 pdtb3_split_level1/Temporal.json
Expansion: 7941 条样本已保存到 pdtb3_split_level1/Expansion.json

✅ 数据集已按类别拆分完成！


In [5]:
import json

def remove_duplicates_by_range(file1, file2, start_line, end_line, output_file):
    """
    从 file2 中删除与 file1 的指定行范围内相同的 JSON。
    :param file1: 第一个 JSON 文件路径（逐行 JSON 格式）
    :param file2: 第二个 JSON 文件路径（逐行 JSON 格式）
    :param start_line: 起始行号（从 1 开始）
    :param end_line: 结束行号（包含）
    :param output_file: 输出文件路径
    """

    # === Step 1. 读取 file1 指定行范围的 JSON 对象 ===
    selected_jsons = set()
    with open(file1, "r", encoding="utf-8") as f1:
        for i, line in enumerate(f1, start=1):
            if start_line <= i <= end_line:
                line = line.strip()
                if line:
                    # 用字符串形式存储，方便精确匹配
                    selected_jsons.add(line)

    print(f"选中 {len(selected_jsons)} 条 JSON 来进行对比。")

    # === Step 2. 从 file2 中过滤掉重复项 ===
    kept = []
    removed = 0
    with open(file2, "r", encoding="utf-8") as f2:
        for line in f2:
            line = line.strip()
            if not line:
                continue
            if line not in selected_jsons:
                kept.append(line)
            else:
                removed += 1

    print(f"已从第二个文件中删除 {removed} 条重复 JSON。")

    # === Step 3. 保存过滤后的结果 ===
    with open(output_file, "w", encoding="utf-8") as out:
        for item in kept:
            out.write(item + "\n")

    print(f"结果已保存到: {output_file}")


# ======== 使用示例 ========
if __name__ == "__main__":
    file1 = "./pdtb3_split_level1/Expansion.json"
    file2 = "./pdtb3_train_level1.jsonl"
    output_file = "pdtb3_train_level1_filter_enhence.jsonl"
    remove_duplicates_by_range(file1, file2, start_line=8, end_line=86, output_file=output_file)


选中 79 条 JSON 来进行对比。
已从第二个文件中删除 79 条重复 JSON。
结果已保存到: pdtb3_train_level1_filter_enhence.jsonl


In [2]:
import json

input_path = "test.jsonl"      # 原始数据文件
output_path = "output.jsonl"    # 处理后输出文件

with open(input_path, 'r', encoding='utf-8') as fin, open(output_path, 'w', encoding='utf-8') as fout:
    for line in fin:
        if not line.strip():
            continue
        data = json.loads(line)
        data["Arg2"] = ""               # Arg2 置空
        data["Label"] = "Temporal"      # 修改标签
        fout.write(json.dumps(data, ensure_ascii=False) + "\n")

print("✅ 处理完成，结果已保存到", output_path)


✅ 处理完成，结果已保存到 output.jsonl
